# Segmentation by registration and feature extraction

In [1]:
import glob
import os
import os.path
import numpy as np
import pandas as pd
import maweight
import pickle
import logging
import tqdm

from config import manually_segmented_path, dissected_path
from config import output_path, mld_features_path, hinds_features_path 
from config import save_registered_images
from config import bin_width, threshold

LIMIT=None

# setting the logging format
FORMAT = '%(asctime)-15s %(clientip)s %(user)-8s %(message)s'
logging.basicConfig(format=FORMAT, level=logging.INFO)

Executables being used: /bin/elastix /bin/transformix


## Discovering files to process

In [2]:
manually_segmented_files= sorted(glob.glob(os.path.join(manually_segmented_path, '*.nii')))
manually_segmented_images= [f for f in manually_segmented_files if not 'hinds' in 
                            f.split(os.sep)[-1] and not 'mld' in f.split(os.sep)[-1]]
manually_segmented_hinds= [f for f in manually_segmented_files if 'hinds' in f]
manually_segmented_mlds= [f for f in manually_segmented_files if 'mld' in f]

if LIMIT is None:
    dissected_images= sorted(glob.glob(os.path.join(dissected_path, '*.nii')))
else:
    dissected_images= sorted(glob.glob(os.path.join(dissected_path, '*.nii')))[:LIMIT]

## Segmentation by Registration

In [3]:
for d in tqdm.tqdm(dissected_images):
    for (i, h, m) in zip(manually_segmented_images, manually_segmented_hinds, manually_segmented_mlds):
        output_mld= os.path.join(output_path, d.split(os.sep)[-1] + '-' + m.split(os.sep)[-1])
        output_hinds= os.path.join(output_path, d.split(os.sep)[-1] + '-' + h.split(os.sep)[-1])
        if save_registered_images:
            output_registered= os.path.join(output_path, d.split(os.sep)[-1] + '-' + i.split(os.sep)[-1])
        else:
            output_registered= None
        
        if (not os.path.isfile(output_mld) or not os.path.isfile(output_hinds) or 
            (save_registered_images and not os.path.isfile(output_registered))):
            maweight.register_and_transform(i, d, [m, h], [output_mld, output_hinds], 
                                          registered_image_path= output_registered, verbose= 0)

  3%|▎         | 5/170 [18:30<10:10:53, 222.14s/it]


KeyboardInterrupt: 

## Extracting the features

In [4]:
def extract_features(dissected_images, manually_segmented_targets):
    dataframes= []
    for d in tqdm.tqdm(dissected_images):
        fitted_masks= []
        for m in manually_segmented_targets:
            output_mld= os.path.join(output_path, d.split(os.sep)[-1] + '-' + m.split(os.sep)[-1])
            fitted_masks.append(output_mld)

        labels= [f.split(os.sep)[-1][14:] for f in fitted_masks]

        dataframes.append(maweight.extract_features_3d(d, fitted_masks, labels, bins=[i*bin_width for i in range(int(200/bin_width)+1)], thresholds=[threshold]))
    dataframes= pd.concat(dataframes, axis=0)
    dataframes['filename']= dissected_images
    
    return dataframes

In [5]:
mld_features= extract_features(dissected_images, manually_segmented_mlds)
mld_features.to_csv(mld_features_path, index=False)

  8%|▊         | 14/170 [01:47<19:59,  7.69s/it]


KeyboardInterrupt: 

In [14]:
hinds_features= extract_features(dissected_images, manually_segmented_hinds)
hinds_features.to_csv(hinds_features_path, index=False)

100%|██████████| 170/170 [20:15<00:00,  7.15s/it]


In [16]:
mld_features.columns

Index(['num-203a-mld.nii-0.500000', 'sum-203a-mld.nii-0.500000',
       'mean-203a-mld.nii-0.500000', 'std-203a-mld.nii-0.500000',
       'skew-203a-mld.nii-0.500000', 'kurt-203a-mld.nii-0.500000',
       'hist-0-203a-mld.nii-0.500000', 'hist-1-203a-mld.nii-0.500000',
       'hist-2-203a-mld.nii-0.500000', 'hist-3-203a-mld.nii-0.500000',
       'hist-4-203a-mld.nii-0.500000', 'hist-5-203a-mld.nii-0.500000',
       'hist-6-203a-mld.nii-0.500000', 'hist-7-203a-mld.nii-0.500000',
       'hist-8-203a-mld.nii-0.500000', 'hist-9-203a-mld.nii-0.500000',
       'num-203k-mld.nii-0.500000', 'sum-203k-mld.nii-0.500000',
       'mean-203k-mld.nii-0.500000', 'std-203k-mld.nii-0.500000',
       'skew-203k-mld.nii-0.500000', 'kurt-203k-mld.nii-0.500000',
       'hist-0-203k-mld.nii-0.500000', 'hist-1-203k-mld.nii-0.500000',
       'hist-2-203k-mld.nii-0.500000', 'hist-3-203k-mld.nii-0.500000',
       'hist-4-203k-mld.nii-0.500000', 'hist-5-203k-mld.nii-0.500000',
       'hist-6-203k-mld.nii-0.500000

In [17]:
mld_features.to_csv(mld_features_path, index=False)
hinds_features.to_csv(hinds_features_path, index=False)